# LangGraph Tutorial: Parallel Execution

**Objective:** Understand how the agent executes multiple independent tasks simultaneously.

**Key Learning:**
- When tasks are INDEPENDENT, the agent calls ALL tools at once
- ToolNode executes tools in parallel using ThreadPoolExecutor
- Results are collected and agent synthesizes single response

**Pattern:**
```
Query: Task A AND Task B (independent)
  ↓
Agent: Calls both tools simultaneously
  ↓
ToolNode: Executes [Tool A, Tool B] in parallel
  ↓
Agent: Synthesizes final response
```

---

## Setup

Build the financial assistant graph with currency converter and EMI calculator.

In [19]:
# Core imports
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv
from typing import Literal

load_dotenv("../../.env")
print("✅ Environment loaded")

✅ Environment loaded


In [20]:
# Define tools
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """Convert currency from one type to another."""
    exchange_rates = {"USD": 1.0, "EUR": 0.92, "GBP": 0.79, "INR": 83.12, "JPY": 149.50}
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    if from_currency not in exchange_rates or to_currency not in exchange_rates:
        return f"Error: Unsupported currency"
    
    amount_in_usd = amount / exchange_rates[from_currency]
    converted_amount = amount_in_usd * exchange_rates[to_currency]
    effective_rate = exchange_rates[to_currency] / exchange_rates[from_currency]
    
    return (
        f"Conversion Result:\n"
        f"  {amount:,.2f} {from_currency} = {converted_amount:,.2f} {to_currency}\n"
        f"  Exchange Rate: 1 {from_currency} = {effective_rate:.4f} {to_currency}"
    )

@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """Calculate the EMI (Equated Monthly Installment) for a loan."""
    if principal <= 0 or annual_interest_rate < 0 or tenure_months <= 0:
        return "Error: Invalid input parameters"
    
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    return (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months\n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )

print("✅ Tools defined")

✅ Tools defined


In [21]:
# Initialize LLM and build graph
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0.3,
    max_tokens=1024,
    project=os.getenv("GOOGLE_PROJECT_ID"),
    location=os.getenv("GOOGLE_REGION")
)

tools = [currency_converter, emi_calculator]
llm_with_tools = llm.bind_tools(tools)

def call_llm(state: MessagesState):
    """Agent node: Calls LLM with current messages."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", END]:
    """Routing logic: Check if agent wants to use tools."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

# Build graph
workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_llm)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "agent")

app = workflow.compile()
print("✅ Graph compiled")

✅ Graph compiled


---

## Parallel Execution Example

**Query:** "Convert 100000 USD to EUR AND ALSO calculate EMI for 500000 INR at 8.5% for 60 months"

**Why Parallel?**
- Currency conversion is INDEPENDENT of EMI calculation
- Neither task needs the other's result
- Agent can call both tools simultaneously

In [29]:
# Create state with user query
state = {
    "messages": [
        HumanMessage(content="Convert 100000 USD to EUR AND ALSO calculate EMI for 500000 INR at 8.5% for 60 months")
    ]
}

print("Initial State:")
print("=" * 80)
print(f"Query: {state['messages'][0].content}")
print(f"Message count: {len(state['messages'])}")
print("=" * 80)

Initial State:
Query: Convert 100000 USD to EUR AND ALSO calculate EMI for 500000 INR at 8.5% for 60 months
Message count: 1


---

## Execute and Observe

Now let's actually run the code and verify this behavior.

In [30]:
# Execute the graph
result = app.invoke(state)

print("Execution Complete!")
print("=" * 80)
print(f"Total messages in final state: {len(result['messages'])}")
print("=" * 80)

Execution Complete!
Total messages in final state: 5


In [31]:
result

{'messages': [HumanMessage(content='Convert 100000 USD to EUR AND ALSO calculate EMI for 500000 INR at 8.5% for 60 months', additional_kwargs={}, response_metadata={}, id='9dc6b479-5db9-4526-8506-3f79d58e8d61'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'emi_calculator', 'arguments': '{"currency": "INR", "principal": 500000, "tenure_months": 60, "annual_interest_rate": 8.5}'}, '__gemini_function_call_thought_signatures__': {'8925a55a-0e7e-41e2-870f-adbd3e9522e1': 'CuoHAY89a1+MFbbKXRhio/wg5/EIkBb0DEvlI/IXpWAorWWNMBdago6Gb+UpEpc5gjPHuRAI3VM+leLHTrgFfpPPe3BK4AI8QCpQUdkIekEUECyMwk3jjyGfqnjOztM0E47H8uczWbpFUxltGG/dioAym+srxZ61OsJWhGv0iw0AXtTUsYM2MpxAHcCW9KQ/GMym5ekpeL8aY4tpYK5D1qRFXdgxVbyt5Ll39bIIQ7Hvs7saw3WAqBa1AJjqjyABp9KsEgepoQBod4Lq4k8Urig40OvIuAffOqNWDVDSRqQ6TW5vWxln5XNINDHZKas1DBYCVCTtzmZgCmHkQ1gKvGZmi9vhEzA6VIl7WWZnbvpmhnPxJD9AkkUFYR5qoygeNWXFcFmmeMI163Erc6EidKAiZf1K2wGp5AxewwADP+huB7ypA6F/8K8Xq87XTtxC+5DLfXaoeVuoD+4VBc2cvBUcrGrWLSu0cVp0E4L5tyt/APj66d2y3E4y

In [32]:
result['messages'][1].tool_calls

[{'name': 'currency_converter',
  'args': {'amount': 100000, 'from_currency': 'USD', 'to_currency': 'EUR'},
  'id': '8925a55a-0e7e-41e2-870f-adbd3e9522e1',
  'type': 'tool_call'},
 {'name': 'emi_calculator',
  'args': {'currency': 'INR',
   'principal': 500000,
   'tenure_months': 60,
   'annual_interest_rate': 8.5},
  'id': '42a3f884-22b6-4dd3-91b7-f58332e45a34',
  'type': 'tool_call'}]

In [33]:
result['messages'][2]

ToolMessage(content='Conversion Result:\n  100,000.00 USD = 92,000.00 EUR\n  Exchange Rate: 1 USD = 0.9200 EUR', name='currency_converter', id='30a862e4-ae95-4979-acae-669843909fdb', tool_call_id='8925a55a-0e7e-41e2-870f-adbd3e9522e1')

In [34]:
result['messages'][3]

ToolMessage(content='EMI Calculation Result:\n  Loan Amount: 500,000.00 INR\n  Interest Rate: 8.5% per annum\n  Tenure: 60 months\n  Monthly EMI: 10,258.27 INR\n  Total Payment: 615,495.94 INR\n  Total Interest: 115,495.94 INR', name='emi_calculator', id='ccef2a72-525a-456f-9ca2-ab28395b9c8d', tool_call_id='42a3f884-22b6-4dd3-91b7-f58332e45a34')

---

## Stream Execution (Real-Time View)

Use `.stream()` to see each step as it happens.

In [35]:
# Reset state and stream execution
state_stream = {
    "messages": [
        HumanMessage(content="Convert 100000 USD to EUR AND ALSO calculate EMI for 500000 INR at 8.5% for 60 months")
    ]
}

print("STREAMING EXECUTION:")
print("=" * 80)

step_count = 0
for event in app.stream(state_stream):
    for node_name, data in event.items():
        step_count += 1
        print(f"\n[Step {step_count}] Node: {node_name}")
        print("-" * 80)
        
        if "messages" in data:
            last_msg = data["messages"][-1]
            
            if isinstance(last_msg, AIMessage) and hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
                if len(last_msg.tool_calls) > 1:
                    print(f"  🚀 PARALLEL EXECUTION: {len(last_msg.tool_calls)} tools called simultaneously")
                    for tc in last_msg.tool_calls:
                        print(f"     • {tc['name']}")
                        print(f"       Arguments: {tc['args']}")
                else:
                    print(f"  🔧 Tool Call: {last_msg.tool_calls[0]['name']}")
                    print(f"     Arguments: {last_msg.tool_calls[0]['args']}")
                    
            elif isinstance(last_msg, ToolMessage):
                print(f"  ✅ Tool executed successfully")
                print(f"     Result preview: {last_msg.content[:100]}...")
                
            elif isinstance(last_msg, AIMessage):
                print(f"  💬 Final response generated")
                print(f"     Response: {last_msg.content[:150]}...")

print("\n" + "=" * 80)
print(f"Total execution steps: {step_count}")
print("=" * 80)

STREAMING EXECUTION:

[Step 1] Node: agent
--------------------------------------------------------------------------------
  🚀 PARALLEL EXECUTION: 2 tools called simultaneously
     • currency_converter
       Arguments: {'amount': 100000, 'to_currency': 'EUR', 'from_currency': 'USD'}
     • emi_calculator
       Arguments: {'tenure_months': 60, 'currency': 'INR', 'annual_interest_rate': 8.5, 'principal': 500000}

[Step 2] Node: tools
--------------------------------------------------------------------------------
  ✅ Tool executed successfully
     Result preview: EMI Calculation Result:
  Loan Amount: 500,000.00 INR
  Interest Rate: 8.5% per annum
  Tenure: 60 m...

[Step 3] Node: agent
--------------------------------------------------------------------------------
  💬 Final response generated
     Response: 100,000 USD is equal to 92,000 EUR. The EMI for a loan of 500,000 INR at an annual interest rate of 8.5% for a tenure of 60 months is 10,258.27 INR....

Total execution steps: 

---

## Key Insights: Parallel Execution

### When Does Parallel Execution Happen?
✅ Tasks are **independent** (neither needs the other's result)
✅ Agent detects this from query language ("AND ALSO", "both", etc.)
✅ All tool_calls included in SINGLE AIMessage

### How ToolNode Executes Parallel Calls
- Uses Python's **ThreadPoolExecutor**
- Each tool runs in separate thread
- Waits for ALL to complete before proceeding
- Returns multiple ToolMessages (one per tool)

### Performance Benefits
- **Faster execution:** Tools run simultaneously
- **Efficient resource use:** No idle waiting
- **Single agent round-trip:** Only 2 LLM calls needed

### Message Count Pattern
```
Parallel execution:
  1. HumanMessage (query)
  2. AIMessage (tool_calls=[Tool1, Tool2])
  3. ToolMessage (Tool1 result)
  4. ToolMessage (Tool2 result)
  5. AIMessage (final response)

Total: 5 messages, 2 agent calls, 1 loop
```

### LLM Intelligence
✅ No hardcoded logic for parallel execution
✅ Agent autonomously determines independence
✅ Based on natural language understanding

---

## ✅ Parallel Execution Complete!

**You Learned:**
- ✅ How to identify parallel execution patterns
- ✅ Internal step-by-step execution flow of `app.invoke()`
- ✅ How ToolNode executes multiple tools simultaneously
- ✅ Message sequence in parallel execution (5 messages)
- ✅ Performance benefits of parallel tool execution

**Next Steps:**
- Notebook 6: Sequential Execution (dependent tasks)
- Notebook 7: Mixed Execution (combination patterns)